In [1]:
class DirectedWeightedGraph:

    def __init__(self):
        self.adj = {}
        self.weights = {}

    def are_connected(self, node1, node2):
        for neighbour in self.adj[node1]:
            if neighbour == node2:
                return True
        return False

    def adjacent_nodes(self, node):
        return self.adj[node]

    def add_node(self, node):
        self.adj[node] = []

    def add_edge(self, node1, node2, weight):
        if node2 not in self.adj[node1]:
            self.adj[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def w(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def number_of_nodes(self):
        return len(self.adj)

In [2]:
import math

class MinHeap:
    length = 0
    data = []

    def __init__(self, L):
        self.data = L
        self.length = len(L)
        self.map = {}
        for i in range(len(L)):
            self.map[L[i].value] = i
        self.build_heap()

    def build_heap(self):
        for i in range(self.length // 2 - 1, -1, -1):
            self.sink(i)

    def sink(self, i):
        smallest_known = i
        if self.left(i) < self.length and self.data[self.left(i)].key < self.data[i].key:
            smallest_known = self.left(i)
        if self.right(i) < self.length and self.data[self.right(i)].key < self.data[smallest_known].key:
            smallest_known = self.right(i)
        if smallest_known != i:
            self.data[i], self.data[smallest_known] = self.data[smallest_known], self.data[i]
            self.map[self.data[i].value] = i
            self.map[self.data[smallest_known].value] = smallest_known
            self.sink(smallest_known)

    def insert(self, element):
        if len(self.data) == self.length:
            self.data.append(element)
        else:
            self.data[self.length] = element
        self.map[element.value] = self.length
        self.length += 1
        self.swim(self.length - 1)

    def insert_elements(self, L):
        for element in L:
            self.insert(element)

    def swim(self, i):
        while i > 0 and self.data[i].key < self.data[self.parent(i)].key:
            self.data[i], self.data[self.parent(i)] = self.data[self.parent(i)], self.data[i]
            self.map[self.data[i].value] = i
            self.map[self.data[self.parent(i)].value] = self.parent(i)
            i = self.parent(i)

    def get_min(self):
        if len(self.data) > 0:
            return self.data[0]
  
    def extract_min(self):
        self.data[0], self.data[self.length - 1] = self.data[self.length - 1], self.data[0]
        self.map[self.data[self.length - 1].value] = self.length - 1
        self.map[self.data[0].value] = 0
        min_element = self.data[self.length - 1]
        self.length -= 1
        self.map.pop(min_element.value)
        self.sink(0)
        return min_element

    def decrease_key(self, value, new_key):
        if new_key >= self.data[self.map[value]].key:
            return
        index = self.map[value]
        self.data[index].key = new_key
        self.swim(index)

    def get_element_from_value(self, value):
        return self.data[self.map[value]]

    def get_key_from_value(self, value):
        return self.data[self.map[value]].key

    def is_empty(self):
        return self.length == 0
    
    def left(self, i):
        return 2 * (i + 1) - 1

    def right(self, i):
        return 2 * (i + 1)

    def parent(self, i):
        return (i + 1) // 2 - 1

    def __str__(self):
        height = math.ceil(math.log(self.length + 1, 2))
        whitespace = 2 ** height
        s = ""
        for i in range(height):
            for j in range(2 ** i - 1, min(2 ** (i + 1) - 1, self.length)):
                s += " " * whitespace
                s += str(self.data[j]) + " "
            s += "\n"
            whitespace = whitespace // 2
        return s

class Element:

    def __init__(self, value, key):
        self.value = value
        self.key = key

    def __str__(self):
        return "(" + str(self.value) + "," + str(self.key) + ")"

In [3]:
gr1 = DirectedWeightedGraph()
for i in range(8):
    gr1.add_node(i)
gr1.add_edge(4,5,35)
gr1.add_edge(4,7,37)
gr1.add_edge(5,7,28)
gr1.add_edge(0,7,16)
gr1.add_edge(1,5,32)
gr1.add_edge(0,4,38)
gr1.add_edge(2,3,17)
gr1.add_edge(1,7,19)
gr1.add_edge(0,2,26)
gr1.add_edge(1,2,36)
gr1.add_edge(1,3,29)
gr1.add_edge(2,7,34)
gr1.add_edge(6,2,40)
gr1.add_edge(3,6,52)
gr1.add_edge(6,0,58)
gr1.add_edge(6,4,93)


In [4]:
def bellman_ford(G, source):
    pred = {} #Predecessor dictionary. Isn't returned, but here for your understanding
    dist = {} #Distance dictionary
    nodes = list(G.adj.keys())

    #Initialize distances
    for node in nodes:
        dist[node] = 99999
    dist[source] = 0

    #Meat of the algorithm
    for _ in range(G.number_of_nodes()):
        for node in nodes:
            for neighbour in G.adj[node]:
                if dist[neighbour] > dist[node] + G.w(node, neighbour):
                    dist[neighbour] = dist[node] + G.w(node, neighbour)
                    pred[neighbour] = node
    return dist

In [5]:
def bellman_ford_approx(G, source, k):
    pred = {} #Predecessor dictionary. Isn't returned, but here for your understanding
    dist = {} #Distance dictionary
    nodes = list(G.adj.keys())
    c = [0]*G.number_of_nodes()

    #Initialize distances
    for node in nodes:
        dist[node] = 99999
    dist[source] = 0

    #Meat of the algorithm
    for i in range(G.number_of_nodes()):
        for node in nodes:
            for neighbour in G.adj[node]:
                if dist[neighbour] > dist[node] + G.w(node, neighbour) and c[node] < k:
                    dist[neighbour] = dist[node] + G.w(node, neighbour)
                    c[node] += 1
                    pred[neighbour] = node
    return dist

In [6]:
def total_dist(dist):
    total = 0
    for key in dist.keys():
        total += dist[key]
    return total


In [7]:
import random

def create_random_complete_graph(n,upper):
    G = DirectedWeightedGraph()
    for i in range(n):
        G.add_node(i)
    for i in range(n):
        for j in range(n):
            if i != j:
                G.add_edge(i,j,random.randint(1,upper))
    return G

def logtocsv(num,results1,results2,filename):
    with open(filename, 'a') as f: #'w' for write, 'a' for append
        f.write(str(num))
        f.write(",")
        f.write(str(results1))
        f.write(",")
        f.write(str(results2))
        f.write('\n')

def timer1(index):
    if __name__ == '__main__':
        import timeit
        print("timing for original {}".format(index))
        return timeit.repeat("bellman_ford(graph, 0)", setup="from __main__ import bellman_ford, graph", repeat=1, number=1) 

def timer2(index):
    if __name__ == '__main__':
        import timeit
        print("timing for approximation {}".format(index))
        return timeit.repeat("bellman_ford_approx(graph, 0, k)", setup="from __main__ import bellman_ford_approx, graph, k", repeat=1, number=1) 


In [8]:
import copy 

i = 1
while (i < 40):
    for x in range(3):
        testGraph = create_random_complete_graph(100,100)

        graph = copy.copy(testGraph)
        k = i
        logtocsv(i, total_dist(bellman_ford(graph, 0)), total_dist(bellman_ford_approx(graph, 0, k)), "bellman_ford_dist.csv") #To switch function to test, changer here and in the method.

    i += 1

# mst = prim1(gr2)
# mst.show()
# gr2 = create_random_graph(100,200)
# gr2.show()
# gr3 = create_random_graph(10,11)
# gr3.show()
# mst2 = prim2(gr3)
# mst2.show()
# mst3 = prim2(gr3)
# mst3.show()

In [9]:
# gr4 = WeightedGraph(10) #testing for disconnected graph
# gr4.add_edge(0,5,6)
# gr4.add_edge(0,4,8)
# gr4.add_edge(0,7,3)
# gr4.add_edge(2,8,1)
# gr4.add_edge(3,6,7)
# gr4.add_edge(4,9,4)
# gr4.add_edge(4,8,0)
# gr4.add_edge(5,8,11)
# gr4.add_edge(5,7,5)
# gr4.add_edge(6,8,10)
# gr4.add_edge(6,9,9)

# gr4.show()

# mst1 = prim2(gr4)
# mst1.show()